In [1]:
import numpy as np
from scipy import stats
import pandas as pd
from matplotlib import pyplot as plt
from pathlib import Path
from bids import BIDSLayout
import nibabel as nb
import nilearn.image as nli
import seaborn as sns
from joblib import Parallel, delayed, parallel_backend
import json

%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 1000)

from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))



/tmp/ipykernel_45107/1537300254.py:19: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
project_root = Path('..')
bids_dir = project_root / 'data/CATD'
derivatives_dir = bids_dir / 'derivatives'

analysis_name = 'summary_tables'
analysis_dir = derivatives_dir / analysis_name
analysis_dir.mkdir(exist_ok=True)
n_dummy=4

In [3]:
analysis_dir

PosixPath('../data/CATD/derivatives/summary_tables')

In [4]:
database_file='./pybids140_public/'
%time layout = BIDSLayout(bids_dir, database_path=database_file)

CPU times: user 11.2 ms, sys: 2.16 ms, total: 13.3 ms
Wall time: 13.2 ms


# Make session level data files

In [5]:
participants = layout.get(return_type='file', scope='raw', extension='tsv', suffix='participants')[0]
sessions = layout.get(return_type='file', scope='raw', extension='tsv', suffix='sessions')

ses_dat = []
for sp in sessions:
    ss = pd.read_csv(sp, sep='\t')
    ss['subject'] = [xm for xm in  sp.split('_') if 'sub' in xm][0].split('-')[-1]
    ses_dat.append(ss)
ses_dat = pd.concat(ses_dat, ignore_index=True)

p_dat = pd.read_csv(participants, sep='\t')
p_dat['subject'] = p_dat.participant_id.str.split('-').str[-1]
ses_dat = p_dat.loc[:, ['subject', 'sex', 'datasharing']].merge(ses_dat, how='right')

In [6]:
ses_dat.to_csv(analysis_dir / 'session_data_ilr.csv', index=None)

# Make qc table
This is a bit of a mess because some of the qc data was generated by reports from an old, semi-broken version of fmriprep group report, some of it was geneerated from newer versions of fmriprep group report. In general, the reports from fmriprep group report should be much more usable now.

In [7]:
run_name = 'fmriprep_v21.0.0'
fmriprep_out = derivatives_dir / 'fmriprep' / run_name
fmriprep_out.exists()

True

In [8]:
# load bids summary table
summary = pd.read_csv(derivatives_dir / 'summary_tables/session_data_ilr.csv')

# load fitlins prep table
prep_df = pd.read_csv(derivatives_dir / 'summary_tables/prep_df.csv')
prep_df['subject'] = prep_df.subject.astype(str)

In [9]:
qc_csvs = sorted((fmriprep_out / 'group' / 'qc').glob('*.csv'))

all_qc = []
for qcf in qc_csvs:
    tmpqc = pd.read_csv(qcf)
    tmpqc['usable'] = False
    tmpqc.loc[(tmpqc.rater == 'dn') & (tmpqc.report == 1), 'usable'] = True
    
    try:
        tmpqc.loc[(tmpqc.rater != 'dn') & (tmpqc.dn_action != 'review'), 'usable'] = True
    except AttributeError:
        pass
    all_qc.append(tmpqc)
all_qc = pd.concat(all_qc, sort=False)

In [10]:
# add manual review
review_results=[
    {'subject': 21250,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': True
    },
    {'subject': 21748, 'run': 1, 'suffix': 'bold', 'session': 'v3', 'task': 'rest', 'usable': True},
    {'subject': 21748,
 'run': 2,
 'suffix': 'bold',
 'session': 'v3',
 'task': 'mid',
 'usable': True},
    {'subject': 22228,
 'run': 2,
 'suffix': 'bold',
 'session': 'v3',
 'task': 'rest',
 'usable': False},
{'subject': 22477,
 'run': 1,
 'suffix': 'bold',
 'session': 'v2o1',
 'task': 'mid',
 'usable': False},
{'subject': 22694,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': False,
 'note': 'Very weird issue with brain mask.'},
    {'subject': 22698,
 'run': 1,
 'suffix': 'bold',
 'session': 'v4',
 'task': 'mid',
 'usable': True},
    {'subject': 22699,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': False,
    'note': 'confirmed bad mask'},
    {'subject': 22699,
 'run': 1,
 'suffix': 'bold',
 'session': 'v4',
 'task': 'rest',
 'usable': True,
    'note': 'Chunk missing from superior aspect of brain mask, but ok otherwise'},
    {'subject': 22749,
 'run': 1,
 'suffix': 'bold',
 'session': 'v6',
 'task': 'rest',
 'usable': False,
    'note': 'weird mask error'},
    {'subject': 23015,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'rest',
 'usable': False},
    {'subject': 23015,
 'run': 1,
 'suffix': 'bold',
 'session': 'v7',
 'task': 'rest',
 'usable': False},
    {'subject': 23017,
 'run': 1,
 'suffix': 'bold',
 'session': 'o8',
 'task': 'rest',
 'usable': False,
    'note': 'dura/skull included in mask around right temporal lobe'},
    {'subject': 23457,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': False,
    'note': 'lots of skull inclusions in brain mask'},
    {'subject': 23457,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'rest',
 'usable': False,
    'note': 'some misses on brain mask, weird flickering of regions over time.'},
    {'subject': 23495,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': False},
    {'subject': 23495,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'rest',
 'usable': False,
    'note': 'dropout + huge SDC distortion trying to correct for it.'},
    {'subject': 23516,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': False,
    'note': 'massive distortion in brainstem/cebeblum, probably a freesurfer failure'},
    {'subject': 23516,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'rest',
 'usable': False,
        'note': 'massive distortion in brainstem/cebeblum, probably a freesurfer failure'},
{'subject': 23516,
 'run': 1,
 'suffix': 'bold',
 'session': 'v2',
 'task': 'rest',
 'usable': False,
 'note': 'massive distortion in brainstem/cebeblum, probably a freesurfer failure'},
   {'subject': 23516,
 'run': 1,
 'suffix': 'bold',
 'session': 'v3',
 'task': 'mid',
 'usable': False,
   'note': 'massive distortion in brainstem/cebeblum, probably a freesurfer failure'},
   {'subject': 23516,
 'run': 1,
 'suffix': 'bold',
 'session': 'v3',
 'task': 'rest',
 'usable': False,
   'note': 'massive distortion in brainstem/cebeblum, probably a freesurfer failure'},
    {'subject': 23519,
 'run': 1,
 'suffix': 'bold',
 'session': 'v8',
 'task': 'rest',
 'usable': False,
    'note': 'lots of skull inclusions in brain mask'},
    {'subject': 23528,
 'run': 1,
 'suffix': 'bold',
 'session': 'v0',
 'task': 'rest',
 'usable': True},
    {'subject': 23540,
 'run': 1,
 'suffix': 'bold',
 'session': 'i5',
 'task': 'mid',
 'usable': True},
    {'subject': 23544,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'rest',
 'usable': True,
    'note': 'brain masks misses a bit on superior aspect'},
    {'subject': 23544,
 'run': 1,
 'suffix': 'bold',
 'session': 'v2',
 'task': 'rest',
 'usable': True},
    {'subject': 23546,
 'run': 1,
 'suffix': 'bold',
 'session': 'i3',
 'task': 'mid',
 'usable': False,
    'note': 'really bad mask, probaly a failed alignment'},
    {'subject': 23546,
 'run': 1,
 'suffix': 'bold',
 'session': 'i3',
 'task': 'rest',
 'usable': False,
    'note': 'almost certainly a failed alignment, maybe a freesurver fail.'},
    {'subject': 23546,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': True,
    'note': 'There is dropout, but it seems usable still.'},
    {'subject': 23546,
 'run': 2,
 'suffix': 'bold',
 'session': 'v4',
 'task': 'mid',
 'usable': True},
    {'subject': 23549,
 'run': 1,
 'suffix': 'bold',
 'session': 'v3',
 'task': 'rest',
 'usable': False,
    'note': 'alignment'},
    {'subject': 23549,
 'run': 1,
 'suffix': 'bold',
 'session': 'v5o12',
 'task': 'rest',
 'usable': True,
    'note': 'Some brain mask miss on superior aspect but still usable.'},
    {'subject': 23550,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': False,
    'note': 'alignment'},
    {'subject': 23550,
 'run': 1,
 'suffix': 'bold',
 'session': 'v5',
 'task': 'mid',
 'usable': True,
    'note': 'Some brain mask miss on superior aspect but still usable.'},
    {'subject': 23554,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': False,
    'note': 'lots of temporal lobe droupout'},
    {'subject': 23564,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': False,
    'note': 'bad anterior distortion, maybe from SDC.'},
    {'subject': 23574,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1i4',
 'task': 'rest',
 'usable': True},
    {'subject': 23607,
 'run': 1,
 'suffix': 'bold',
 'session': 'v3',
 'task': 'rest',
 'usable': True,
    'note': 'mask missed a little bit superior.'},
    {'subject': 23641,
 'run': 1,
 'suffix': 'bold',
 'session': 'v3',
 'task': 'mid',
 'usable': False,
    'note': 'mask includes skull around temporal lobe.'},
    {'subject': 23644,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'rest',
 'usable': False},
    {'subject': 23644,
 'run': 1,
 'suffix': 'bold',
 'session': 'v2',
 'task': 'rest',
 'usable': False},
    {'subject': 23644,
 'run': 1,
 'suffix': 'bold',
 'session': 'v4',
 'task': 'rest',
 'usable': False},
    {'subject': 23656,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'rest',
 'usable': True,
    'note': 'mask missed a little bit superior.'},
    {'subject': 23672,
 'run': 1,
 'suffix': 'bold',
 'session': 'v6',
 'task': 'rest',
 'usable': True,
    'note': 'mask cuts out a bit, but there is not good signal there'},
    {'subject': 23717,
 'run': 1,
 'suffix': 'bold',
 'session': 'v2',
 'task': 'mid',
 'usable': True,
    'note': 'mask missed a little bit superior.'},
    {'subject': 23732,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'rest',
 'usable': True},
    {'subject': 23732,
 'run': 1,
 'suffix': 'bold',
 'session': 'v4',
 'task': 'rest',
 'usable': False,
    'note': 'failed alignment'},
    {'subject': 23757,
 'run': 1,
 'suffix': 'bold',
 'session': 'v4',
 'task': 'rest',
 'usable': False,
    'note': 'massive SDC failure'},
    {'subject': 23759,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': False},
    {'subject': 23759,
 'run': 1,
 'suffix': 'bold',
 'session': 'v3',
 'task': 'mid',
 'usable': False},
    {'subject': 23765,
 'run': 1,
 'suffix': 'bold',
 'session': 'v6',
 'task': 'rest',
 'usable': True,
    'note': 'mask missed a little bit superior.'},
    {'subject': 23767,
 'run': 1,
 'suffix': 'bold',
 'session': 'o4',
 'task': 'mid',
 'usable': True},
    {'subject': 23770,
 'run': 1,
 'suffix': 'bold',
 'session': 'v2',
 'task': 'rest',
 'usable': True},
    {'subject': 23770,
 'run': 2,
 'suffix': 'bold',
 'session': 'v2',
 'task': 'mid',
 'usable': True},
    {'subject': 23783,
 'run': 1,
 'suffix': 'bold',
 'session': 'o4',
 'task': 'rest',
 'usable': True,
    'note': 'mask missed occipital pole'},
    {'subject': 23783,
 'run': 1,
 'suffix': 'bold',
 'session': 'v2',
 'task': 'mid',
 'usable': True,
    'note': 'mask missed a little bit superior.'},
    {'subject': 23810,
 'run': 2,
 'suffix': 'bold',
 'session': 'v2',
 'task': 'mid',
 'usable': False,
    'note': 'alignment fail'},
    {'subject': 23810,
 'run': 1,
 'suffix': 'bold',
 'session': 'v5o8',
 'task': 'rest',
 'usable': True,
    'note': 'mask missed a little of occipital pole'},
    {'subject': 23813,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': True,
    'note': 'mask missed a little of occipital pole'},
    {'subject': 23813,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'rest',
 'usable': True,
    'note': 'mask missed a little of occipital pole'},
    {'subject': 23813,
 'run': 1,
 'suffix': 'bold',
 'session': 'v2',
 'task': 'mid',
 'usable': True,
    'note': 'mask missed a little of occipital pole'},
    {'subject': 23813,
 'run': 1,
 'suffix': 'bold',
 'session': 'v3',
 'task': 'mid',
 'usable': True,
    'note': 'mask missed a little of occipital pole'},
    {'subject': 23814,
 'run': 1,
 'suffix': 'bold',
 'session': 'v3',
 'task': 'rest',
 'usable': True,
    'note': 'mask missed a little of occipital pole'},
    {'subject': 23823,
 'run': 1,
 'suffix': 'bold',
 'session': 'i5',
 'task': 'rest',
 'usable': False,
    'note': 'alignment fail'},
    {'subject': 23823,
 'run': 1,
 'suffix': 'bold',
 'session': 'i9',
 'task': 'rest',
 'usable': True,
    'note': 'mask missed a bit of occipital lobe'},
    {'subject': 23823,
 'run': 1,
 'suffix': 'bold',
 'session': 'v4',
 'task': 'rest',
 'usable': True},
    {'subject': 23830,
 'run': 1,
 'suffix': 'bold',
 'session': 'o8',
 'task': 'mid',
 'usable': True,
    'note': 'mask missed right temporal pole'},
    {'subject': 23830,
 'run': 1,
 'suffix': 'bold',
 'session': 'o8',
 'task': 'rest',
 'usable': True},
    {'subject': 23830,
 'run': 1,
 'suffix': 'bold',
 'session': 'v2o12',
 'task': 'mid',
 'usable': True},
    {'subject': 23849,
 'run': 1,
 'suffix': 'bold',
 'session': 'v4',
 'task': 'mid',
 'usable': False,
    'note': 'alignment failure'},
    {'subject': 23880,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': False,
    'note': 'mask also includes ap smears'},
    {'subject': 23895,
 'run': 2,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': True},
    {'subject': 23907,
 'run': 1,
 'suffix': 'bold',
 'session': 'v3',
 'task': 'rest',
 'usable': False,
    'note': 'mask/alignment failure'},
    {'subject': 23924,
 'run': 1,
 'suffix': 'bold',
 'session': 'v4',
 'task': 'rest',
 'usable': True,
    'note': 'mask missed a little bit superior.'},
    {'subject': 23951,
 'run': 1,
 'suffix': 'bold',
 'session': 'i4',
 'task': 'rest',
 'usable': True,
    'note': 'artifacts not visible in nifti'},
    {'subject': 23952,
 'run': 1,
 'suffix': 'bold',
 'session': 'v3',
 'task': 'mid',
 'usable': True,
    'note': 'artifacts not visibile in nifti'},
    {'subject': 23952,
 'run': 1,
 'suffix': 'bold',
 'session': 'v5',
 'task': 'rest',
 'usable': True,
    'note': 'mask missed near right temporal lobe'},
    {'subject': 23988,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': True,
    'note': 'looks fine'},
    {'subject': 23988,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'rest',
 'usable': True,
    'note': 'some weird cerebellum/brainstem stretching'},
    {'subject': 23988,
 'run': 2,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': False},
    {'subject': 23988,
 'run': 1,
 'suffix': 'bold',
 'session': 'v2',
 'task': 'mid',
 'usable': True},
    {'subject': 23988,
 'run': 2,
 'suffix': 'bold',
 'session': 'v2',
 'task': 'mid',
 'usable': False,
    'note': 'truncated time series'},
    {'subject': 23988,
 'run': 3,
 'suffix': 'bold',
 'session': 'v2',
 'task': 'mid',
 'usable': False,
    'note': 'truncated time series'},
    {'subject': 23988,
 'run': 1,
 'suffix': 'bold',
 'session': 'v2',
 'task': 'rest',
 'usable': True},
    {'subject': 23988,
 'run': 1,
 'suffix': 'bold',
 'session': 'v3',
 'task': 'mid',
 'usable': True},
    {'subject': 23988,
 'run': 1,
 'suffix': 'bold',
 'session': 'v3',
 'task': 'rest',
 'usable': True},
    {'subject': 23988,
 'run': 1,
 'suffix': 'bold',
 'session': 'v4',
 'task': 'mid',
 'usable': True},
    {'subject': 23988,
 'run': 1,
 'suffix': 'bold',
 'session': 'v4',
 'task': 'rest',
 'usable': True},
    {'subject': 24014,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': False},
    {'subject': 24014,
 'run': 1,
 'suffix': 'bold',
 'session': 'v4',
 'task': 'mid',
 'usable': False,
    'note': 'mask also includes ap smears'},
    {'subject': 24014,
 'run': 1,
 'suffix': 'bold',
 'session': 'v4',
 'task': 'rest',
 'usable': True},
    {'subject': 24048,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1o1',
 'task': 'mid',
 'usable': True},
    {'subject': 24052,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'rest',
 'usable': True,
    'note': 'artifacts not visible on nifti'},
    {'subject': 24054,
 'run': 1,
 'suffix': 'bold',
 'session': 'v3',
 'task': 'rest',
 'usable': False},
    {'subject': 24057,
 'run': 1,
 'suffix': 'bold',
 'session': 'v2',
 'task': 'mid',
 'usable': False},
    {'subject': 24057,
 'run': 2,
 'suffix': 'bold',
 'session': 'v2',
 'task': 'mid',
 'usable': False},
    {'subject': 24057,
 'run': 1,
 'suffix': 'bold',
 'session': 'v3o4',
 'task': 'mid',
 'usable': False},
    {'subject': 24057,
 'run': 1,
 'suffix': 'bold',
 'session': 'v3o4',
 'task': 'rest',
 'usable': False},
    {'subject': 24059,
 'run': 1,
 'suffix': 'bold',
 'session': 'v2o12',
 'task': 'rest',
 'usable': True,
    'note': 'artifact not apparent in nifti.'},
    {'subject': 24065,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': False},
    {'subject': 24077,
 'run': 2,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': False,
    'note': 'SDC is not great',},
    {'subject': 24088,
 'run': 1,
 'suffix': 'bold',
 'session': 'v2',
 'task': 'mid',
 'usable': True,
    'note': 'mask misses frontal regions'},
    {'subject': 24092,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': True,
    'note': 'temporal lobe dropout'},
    {'subject': 24092,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'rest',
 'usable': True},
    {'subject': 24093,
 'run': 1,
 'suffix': 'bold',
 'session': 'o8',
 'task': 'rest',
 'usable': False,
    'note': 'alignment failure'},
    {'subject': 24107,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': True},
    {'subject': 24111,
 'run': 1,
 'suffix': 'bold',
 'session': 'v1',
 'task': 'mid',
 'usable': False,
    'note': 'alignment failure'}
]
review_results = pd.DataFrame(review_results)
review_results['reviewed'] = True

In [11]:
all_qc_rev = all_qc.loc[all_qc.run.notnull()].merge(review_results, how='left', on=['subject', 'run', 'suffix', 'session', 'task'], suffixes=['', '_rev'])
all_qc_rev['reviewed'] = all_qc_rev.reviewed.fillna(False)

In [12]:
# add in qc from additional runs
fmapaffinefix_qc = [pd.read_csv(ff, sep='\t') for ff in sorted((derivatives_dir / 'fmriprep/fmriprep_v21.0.0/group/fmapaffinefix/qc-reports').glob('*.tsv'))]
fmapaffinefix_qc = pd.concat(fmapaffinefix_qc)
fmapaffinefix_qc['usable'] = fmapaffinefix_qc.report == 1

all_qc_rev_fmap = all_qc_rev.merge(fmapaffinefix_qc, how='outer', on=['subject', 'session', 'acquisition',
                                                                      'reconstruction', 'run', 'suffix',
                                                                      'space', 'desc', 'task',
                                                                      'report_type'], suffixes=['', '_fmap'], indicator='_fmap')
in_fmap_qc = all_qc_rev_fmap._fmap != 'left_only'

all_qc_rev_fmap.loc[in_fmap_qc, 'usable'] = all_qc_rev_fmap.loc[in_fmap_qc, 'usable_fmap']
all_qc_rev_fmap.loc[in_fmap_qc, 'note'] = all_qc_rev_fmap.loc[in_fmap_qc, 'note_fmap']
all_qc_rev_fmap.loc[in_fmap_qc, 'report'] = all_qc_rev_fmap.loc[in_fmap_qc, 'report_fmap']
all_qc_rev_fmap.loc[in_fmap_qc, 'been_on_screen'] = all_qc_rev_fmap.loc[in_fmap_qc, 'been_on_screen_fmap']
all_qc_rev_fmap.loc[in_fmap_qc, 'rater'] = all_qc_rev_fmap.loc[in_fmap_qc, 'rater_fmap']
all_qc_rev_fmap['from_fmap_corrected_run'] = all_qc_rev_fmap._fmap != 'left_only'
all_qc_rev_fmap = all_qc_rev_fmap.drop(['usable_fmap', 'note_fmap', 'report_fmap', 'been_on_screen_fmap', 'rater_fmap', '_fmap'], axis=1)

# update based on reviews
all_qc_rev_fmap.loc[all_qc_rev_fmap.usable_rev.notnull(), 'usable'] = all_qc_rev_fmap.loc[all_qc_rev_fmap.usable_rev.notnull(), 'usable_rev']

# fix bool casting
assert np.all([isinstance(val, bool) for val in all_qc_rev_fmap.usable.unique()])
old_pct = all_qc_rev_fmap.usable.mean()
all_qc_rev_fmap['usable'] = all_qc_rev_fmap.usable.astype(bool)
new_pct = all_qc_rev_fmap.usable.mean()
assert old_pct == new_pct

In [13]:
all_qc_rev_fmap.groupby(['report_type']).usable.mean()

report_type
MNI152NLin2009cAsym    1.000000
bbregister             0.960000
dseg                   1.000000
sdc                    0.956859
Name: usable, dtype: float64

In [14]:
all_qc = all_qc_rev_fmap.copy()

In [15]:
all_qc.to_csv(derivatives_dir / 'summary_tables/all_qc_fmriprep_v21.0.0.csv', index=None)

In [16]:

def extract_ex_vars(row, n_dummy=4):
    cfd_ents = json.loads(row.entities.replace("'", '"'))
    cfd = pd.read_csv(row.cfd_new_path, sep='\t')
    cfd.censored = cfd.censored.astype(bool)

    nd_cfd = cfd.loc[n_dummy:, :]

    cfd_ents['motion_per_tr'] = nd_cfd.framewise_displacement.mean()
    cfd_ents['motion_per_tr_pc'] = nd_cfd.loc[~nd_cfd.censored, 'framewise_displacement'].mean()

    cfd_ents['max_fd'] = nd_cfd.framewise_displacement.max()
    cfd_ents['max_fd_pc'] = nd_cfd.loc[~nd_cfd.censored, 'framewise_displacement'].max()

    cfd_ents['pct_censored'] = nd_cfd.censored.mean()
    cfd_ents['nv'] = row.nv
    cfd_ents['ms'] = row.ms
    cfd_ents['has_phys'] = row.has_phys
    return pd.Series(cfd_ents)

In [17]:
ex_vars = prep_df.apply(extract_ex_vars, axis=1)

In [18]:
# merge in bbr and sdc qc
bbrs = all_qc.loc[all_qc.report_type == 'bbregister', ['subject', 'run', 'suffix', 'session', 'task', 'note', 'usable', 'dn_action']]
sdcs = all_qc.loc[all_qc.report_type == 'sdc', ['subject', 'run', 'suffix', 'session', 'task', 'note', 'usable', 'dn_action']]
tmp = bbrs.merge(sdcs, how='outer', on=['subject', 'run', 'suffix', 'session', 'task',], suffixes=['_bbr', '_sdc'], indicator=True)
unpaired_qc = tmp.query("_merge != 'both'")
print(f'{len(unpaired_qc)} sessions from {unpaired_qc.subject.nunique()} subjects are missing either a bbr or sdc report.')
mrg_qc = bbrs.merge(sdcs, how='inner', on=['subject', 'run', 'suffix', 'session', 'task',], suffixes=['_bbr', '_sdc'], indicator=False)
mrg_qc['usable'] = mrg_qc.usable_sdc & mrg_qc.usable_bbr

# merge in other exculsionary metrics
mrg_qc['subject'] = mrg_qc.subject.astype(str)
tmp = mrg_qc.merge(ex_vars, how='outer', on=['subject', 'session', 'task', 'run', 'suffix'], indicator=True)
non_mergers = tmp.loc[tmp._merge != 'both', ['subject', 'session', 'task', 'run', '_merge']].subject.unique()
print(f"These subjects don't merge: {non_mergers}.")

mrg_qc = mrg_qc.merge(ex_vars, how='inner', on=['subject', 'session', 'task', 'run', 'suffix'], indicator=False)

21 sessions from 13 subjects are missing either a bbr or sdc report.
These subjects don't merge: ['23546' '23823' '23830' '21670' '22228' '22477' '23550' '23554' '23574'
 '23638' '23652' '23660' '23766' '23810' '23849' '23988'].


In [19]:
mrg_qc['exclude'] = ((mrg_qc.max_fd >= 10) 
                        | (mrg_qc.pct_censored >= 0.15)
                        | (mrg_qc.motion_per_tr_pc >= 0.3)
                        | (~mrg_qc.usable))

In [20]:
mrg_qc.to_csv(derivatives_dir / 'summary_tables/merged_qc_fmriprep_v21.0.0.csv', index=None)

In [21]:
# prep clinical info
short_summary = summary.loc[:, ['subject', 'session_id', 'sex', 'group', 'age', 'scanner', 'inpatient', 'acq_date',
                                'c_ksadsdx_date', 'c_ksadsdx_dx_detailed',
                                's_mfq_date', 's_mfq_tot', 's_mfq_complete', 
                                's_shaps_date', 's_shaps_tot', 's_shaps_binary_tot','s_shaps_complete',
                                's_ari1w_date', 's_ari1w_tot', 's_ari1w_complete',
                                's_scared_date', 's_scared_tot', 's_scared_complete',
                               'antidepressants',
                             'antipsychotics',
                             'anticonvulsants',
                             'ADHD_medication',
                             'antianxiety',
                             'other',
                             'no_meds',
                             'med_source']].copy()

short_summary['acq_date'] = pd.to_datetime(short_summary.acq_date)
for cc in short_summary.columns[short_summary.columns.str.contains('date')]:
    if cc != 'acq_date':
        offset_col_name = cc.replace('_date', '_offset')
        short_summary[offset_col_name] = (pd.to_datetime(short_summary[cc]) - short_summary.acq_date).dt.days
        short_summary = short_summary.drop(cc, axis=1)
short_summary = short_summary.loc[:, ['subject', 'session_id', 'sex', 'group', 'age', 'scanner', 'inpatient', 'acq_date',
                                'c_ksadsdx_offset', 'c_ksadsdx_dx_detailed',
                                's_mfq_offset', 's_mfq_tot', 's_mfq_complete', 
                                's_shaps_offset', 's_shaps_tot', 's_shaps_binary_tot','s_shaps_complete',
                                's_ari1w_offset', 's_ari1w_tot', 's_ari1w_complete',
                                's_scared_offset', 's_scared_tot', 's_scared_complete',
                               'antidepressants',
                             'antipsychotics',
                             'anticonvulsants',
                             'ADHD_medication',
                             'antianxiety',
                             'other',
                             'no_meds',
                             'med_source']].copy()
short_summary['subject'] = short_summary.subject.astype(str)

In [22]:
#merge qc and clinical info
mrg_qc['session_id'] = 'ses-' + mrg_qc['session']

In [23]:
tmp = mrg_qc.merge(short_summary, how='outer', on=['subject', 'session_id'], indicator=True)
# make sure there aren't any subjects that are in the qc data but not in the clinical data
assert (tmp._merge != 'left_only').all()
clin_only = tmp.query('_merge != "both"')
print(f'{len(clin_only)} sessions from {clin_only.subject.nunique()} subjects are not in mrg_qc.')
clin_qc = mrg_qc.merge(short_summary, how='inner', on=['subject', 'session_id'], indicator=False)

38 sessions from 4 subjects are not in mrg_qc.


In [24]:
clin_qc['dropout'] = False
clin_qc.loc[clin_qc.note_bbr.str.contains('do') | clin_qc.dn_action_bbr.str.contains('do'), 'dropout'] = True
clin_qc['sub_ses'] = clin_qc.subject.astype(str) + '_' + clin_qc.session

In [25]:
multi_run_keeps = [
    ('23843_v4', 2),
    ('24107_v1', 2),
    ('24174_v1', 1),
    ('22892_v2', 2),
    ('23520_i9', 2),
    ('23574_v1i4', 2),
    ('23641_v1', 2),
    ('23689_v4', 2),
    ('23746_v1', 2)
]
for ss, rn in multi_run_keeps:
    clin_qc = clin_qc.loc[(clin_qc.sub_ses != ss) | (clin_qc.run == rn)].copy()
multi_med_keeps = [
    ('24065_o4', 'medsclin1yr'),
]
for ss, ms in multi_med_keeps:
    clin_qc = clin_qc.loc[(clin_qc.sub_ses != ss) | (clin_qc.med_source == ms)].copy()

In [26]:
usable_rest_no_mfq_thresh = clin_qc.query("task == 'rest' & ~exclude ", engine='python')

In [27]:
tmp = usable_rest_no_mfq_thresh.merge(prep_df.query('task == "rest"'),
                                   how='left',
                                   on=['subject', 'session', 'run', 'task', 'nv', 'ms', 'has_phys', 'suffix',
                                      'space', 'datatype', 'extension'],
                                   indicator=True)
assert len(tmp.query("_merge != 'both'")) == 0
rest_sessamplenothresh_prep = usable_rest_no_mfq_thresh.merge(prep_df.query('task == "rest"'),
                                   how='left',
                                   on=['subject', 'session', 'run', 'task', 'nv', 'ms', 'has_phys', 'suffix',
                                      'space', 'datatype', 'extension'],
                                   )


In [28]:
n_ses_dict = usable_rest_no_mfq_thresh.groupby('subject').sub_ses.nunique().to_dict()

rest_sessamplenothresh_prep['n_sessions'] = rest_sessamplenothresh_prep.subject.astype(str)
rest_sessamplenothresh_prep['n_sessions'] = rest_sessamplenothresh_prep.n_sessions.replace(n_ses_dict)
assert len(rest_sessamplenothresh_prep.loc[rest_sessamplenothresh_prep.n_sessions.isnull()]) == 0

In [29]:
n_sess_info = rest_sessamplenothresh_prep.groupby('subject').first().groupby(['group']).n_sessions.describe()
n_sess_info['n_ses'] = n_sess_info['count'] * n_sess_info['mean']
n_sess_info = n_sess_info.rename(columns={'count':'n_subj'})
n_sess_info.reset_index().set_index(['n_subj', 'n_ses']).reset_index().set_index(['group'])

,n_subj,n_ses,mean,std,min,25%,50%,75%,max
group,,,,,,,,,
HV,77.0,118.0,1.532468,0.735999,1.0,1.0,1.0,2.0,5.0
MDD,101.0,383.0,3.792079,1.986539,1.0,2.0,4.0,5.0,10.0


In [30]:
rest_sessamplenothresh_prep.to_csv(derivatives_dir / 'summary_tables' / 'rest_sessamplenothresh_prep.csv', index=None)